In [2]:
import pandas as pd

from datetime import datetime
from sklearn import datasets, ensemble, model_selection

from evidently.dashboard import Dashboard
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.tabs import ProbClassificationPerformanceTab

from evidently.model_profile import Profile
from evidently.profile_sections import ProbClassificationPerformanceProfileSection

# Breast Cancer Data

In [3]:
bcancer = datasets.load_breast_cancer()

In [4]:
bcancer_frame = pd.DataFrame(bcancer.data, columns = bcancer.feature_names)

In [5]:
bcancer_frame['target'] = bcancer.target

In [6]:
bcancer_frame.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [7]:
target = 'target'
prediction = 'prediction'

numerical_features = bcancer.feature_names
categorical_features = []

features = numerical_features.tolist() + categorical_features

# Probabilistic Model Performance Dashboard

In [8]:
train_data, test_data = model_selection.train_test_split(bcancer_frame, random_state=0)

In [9]:
model = ensemble.RandomForestClassifier(random_state=0)

In [10]:
model.fit(train_data[features], train_data.target)

RandomForestClassifier(random_state=0)

In [11]:
# make a data frame with predictions for each class

train_probas = pd.DataFrame(model.predict_proba(train_data[features]))
train_probas.columns = bcancer.target_names
test_probas = pd.DataFrame(model.predict_proba(test_data[features]))
test_probas.columns = bcancer.target_names

In [12]:
train_probas.head()

,malignant,benign
0,0.00,1.00
1,0.00,1.00
2,1.00,0.00
3,0.01,0.99
4,0.90,0.10


In [13]:
# get labels for target: [0, 1, 0, 2] -> ['setosa', 'versicolor', 'setosa', 'virginica']

train_data['target'] = [bcancer.target_names[x] for x in train_data['target']]
test_data['target'] = [bcancer.target_names[x] for x in test_data['target']]

In [14]:
# merge train and test data with predictions

train_data.reset_index(inplace=True, drop=True)
test_data.reset_index(inplace=True, drop=True)

merged_train_data = pd.concat([train_data, train_probas], axis=1)
merged_test_data = pd.concat([test_data, test_probas], axis=1)

In [15]:
merged_train_data.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target,malignant,benign
0,11.85,17.46,75.54,432.7,0.08372,0.05642,0.026880,0.022800,0.1875,0.05715,...,517.8,0.1369,0.17580,0.13160,0.09140,0.3101,0.07007,benign,0.00,1.00
1,11.22,19.86,71.94,387.3,0.10540,0.06779,0.005006,0.007583,0.1940,0.06028,...,436.1,0.1424,0.09669,0.01335,0.02022,0.3292,0.06522,benign,0.00,1.00
2,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.144000,0.097910,0.1752,0.05533,...,1731.0,0.1166,0.19220,0.32150,0.16280,0.2572,0.06637,malignant,1.00,0.00
3,13.59,17.84,86.24,572.3,0.07948,0.04052,0.019970,0.012380,0.1573,0.05520,...,739.1,0.1050,0.07622,0.10600,0.05185,0.2335,0.06263,benign,0.01,0.99
4,16.69,20.20,107.10,857.6,0.07497,0.07112,0.036490,0.023070,0.1846,0.05325,...,1084.0,0.1009,0.29200,0.24770,0.08737,0.4677,0.07623,malignant,0.90,0.10


In [16]:
bcancer_column_mapping = ColumnMapping()

bcancer_column_mapping.target = target
bcancer_column_mapping.prediction = bcancer.target_names.tolist()
bcancer_column_mapping.numerical_features = numerical_features

## Probabilistic Model Performance Dashboard full (verbose_level=1)

In [17]:
bcancer_model_performance_dashboard = Dashboard(tabs=[ProbClassificationPerformanceTab(verbose_level=1)])
bcancer_model_performance_dashboard.calculate(merged_train_data, merged_test_data, 
                                              column_mapping = bcancer_column_mapping)
bcancer_model_performance_dashboard.show()

In [18]:
# bcancer_model_performance_dashboard.save('bcancer_model_performance.html')

## Probabilistic Model Performance Dashboard short (verbose_level=0)

In [19]:
bcancer_model_performance_dashboard = Dashboard(tabs=[ProbClassificationPerformanceTab(verbose_level=0)])
bcancer_model_performance_dashboard.calculate(merged_train_data, merged_test_data, 
                                              column_mapping = bcancer_column_mapping)
bcancer_model_performance_dashboard.show()

In [20]:
# bcancer_model_performance_dashboard.save('bcancer_model_performance.html')

# Model Performance Profile

In [21]:
bcancer_classification_performance_profile = Profile(sections=[ProbClassificationPerformanceProfileSection()])

In [22]:
bcancer_classification_performance_profile.calculate(merged_train_data, merged_test_data, 
                                                     column_mapping = bcancer_column_mapping)

In [23]:
bcancer_classification_performance_profile.json() 

'{"probabilistic_classification_performance": {"name": "probabilistic_classification_performance", "datetime": "2022-01-03 12:31:07.907545", "data": {"utility_columns": {"date": null, "id": null, "target": "target", "prediction": ["malignant", "benign"]}, "cat_feature_names": [], "num_feature_names": ["mean radius", "mean texture", "mean perimeter", "mean area", "mean smoothness", "mean compactness", "mean concavity", "mean concave points", "mean symmetry", "mean fractal dimension", "radius error", "texture error", "perimeter error", "area error", "smoothness error", "compactness error", "concavity error", "concave points error", "symmetry error", "fractal dimension error", "worst radius", "worst texture", "worst perimeter", "worst area", "worst smoothness", "worst compactness", "worst concavity", "worst concave points", "worst symmetry", "worst fractal dimension"], "target_names": null, "metrics": {"reference": {"accuracy": 1.0, "precision": 1.0, "recall": 1.0, "f1": 1.0, "roc_auc": 1